### 使用 Flask 撰寫一個會員系統，將會員資料存入 MongoDB，並提供 4 種 API 操作 MongoDB 中的資料，分別是新增、修改、刪除、查詢

### 1. 查詢、新增、刪除、修改 mongo db 

In [ ]:
from flask import Flask, request, jsonify
from flask_pymongo import PyMongo
from bson.objectid import ObjectId

# 建立 Flask 物件
app = Flask(__name__)

# 設定連線 mongo server 的資訊
app.config['MONGO_URI'] = 'mongodb://localhost:27017/test' # mongo資料庫的位址 (test => db name)
mongo = PyMongo(app) 

# 1.新增 mongo db 資料 
@app.route('/members', methods = ['POST']) # 新增會員資料 (使用 POST)
def insert_member() :
    # 使用 postman 工具，在 form-data 填入以下的資料 
    # 把 form 裡面的資料拿出來
    name = request.form.get('name')  
    gender = request.form.get('gender')
    email = request.form.get('email')
    
    result = mongo.db.member.insert_one({'name':name, 'gender':gender, 'email':email})

    return str(result.inserted_id)
    # 把此新增id貼到網址localhost:5000/members/後面 =>回傳 json格式的新增資料

# 2.查詢 mongo db 資料 =>
@app.route('/members') # 取得所有會員資料 
@app.route('/members/<id>', methods = ['GET']) # 只取得所有某會員資料
def find_member(id=None) :
    # 若沒有傳入 id，就取得所有會員資料
    if id is None :
        members = mongo.db.member.find() 
        result = []

        for member in members :  
            member['_id'] = str(member['_id']) # mongo回傳的 id 為 ObjectId 型別，需轉成 string 型別
            result.append(member)

        return jsonify(result) # 將結果轉為 json 格式並回傳 
    
    # 若有傳入 id，只取得此 id 會員資料
    else :
        result = mongo.db.member.find_one({'_id':ObjectId(id)}) 

        if result is not None :
            result['_id'] = str(result['_id']) 

        return jsonify(result) # 將結果轉為 json 格式並回傳

# 3.刪除 mongo db 資料 =>
@app.route('/members/<id>', methods = ['DELETE']) # 使用 postman 工具，貼上一筆要刪除資料的 _id值
def delete_member(id=None) :
    result = 0
    
    # 確認是否有此會員
    member = mongo.db.member.find_one({'_id':ObjectId(id)})

    # 若有此會員，則刪除此筆資料
    if member is not None :
        del_result = mongo.db.member.delete_one(member)
        result = del_result.deleted_count 
        
    # 回傳刪除的資料筆數
    return 'deleted data count: {}' .format(str(result))

# 4.修改 mongo db 資料 => 
@app.route('/members/<id>', methods = ['PUT']) # 使用 postman 工具，貼上一筆要修改資料的 _id值
def update_member(id=None) :
    result = 0
    
    # 使用 postman 工具，在 form-data 填入以下的資料 
    # 把 form 裡面的資料拿出來
    name = request.form.get('name')  
    gender = request.form.get('gender')
    email = request.form.get('email')
    
    # 1.過濾條件 => 哪些資料要被修改
    filter_ = {'_id':ObjectId(id)}
    # 2.要改成什麼資料
    update_ = {'$set':{'name':name, 'gender':gender, 'email':email}}
    upd_result = mongo.db.member.update_one(filter_, update_)

    # 若有此會員，則修改此筆資料
    if upd_result is not None :
        result = upd_result.modified_count 
     
    # 回傳修改的資料筆數
    return 'updated data count: {}' .format(str(result))
    
if __name__ == '__main__' :
    # default 設定值 => app.run(host='127.0.0.1', port=5000)
    app.run()

#### 1.1 打開 postman 新增 db 資料

In [ ]:
# **在 test(db) 下新增一筆會員資料：

1. postman -> my workspace -> new tab
2. POST -> localhost:5000/members
3. Body -> form-data -> 填上資料 -> send
4. response -> (回傳為新增 _id 內容)

![insert_db_members](img/insert_db_members.png)

In [ ]:
# **查詢 test(db) 下新增的一筆會員資料：

1. GET -> localhost:5000/members/ (新增資料的 _id值，貼在網址後面，再發送 request 網址) -> send  
例：localhost:5000/members/601bc76ab34a87b4dc757280

![insert_db_members_id](img/insert_db_members_id.png)

#### 1.2 打開 postman 查詢 db 資料

In [ ]:
# **查詢 test(db) 下的所有會員資料：

1. postman -> my workspace -> new tab
2. GET -> localhost:5000/members -> send  

![find_db_members](img/find_db_members.png)

In [ ]:
# **查詢 test(db) 下的某 id 會員資料：

1. GET -> localhost:5000/members/ (選擇任意一筆資料 _id值，貼在網址後面，再發送 request 網址) -> send  
例：localhost:5000/members/6014278203a07fa5e37dccbb

![find_db_members_id](img/find_db_members_id.png)

#### 1.3 打開 postman 刪除 db 資料

In [ ]:
# **在 test(db) 下刪除一筆會員資料：

1. postman -> my workspace -> new tab
2. DELETE -> localhost:5000/members/ (選擇任意一筆要刪除的 _id值，貼在網址後面，再發送 request 網址) -> send 
例：localhost:5000/members/601cb873eda3eb882a35270b
3. response -> deleted data count: 1 (回傳要刪除資料的筆數)

![delete_db_members_id](img/delete_db_members_id.png)

#### 1.4 打開 postman 修改 db 資料

In [ ]:
# **在 test(db) 下修改一筆會員資料：

1. postman -> my workspace -> new tab
2. PUT -> localhost:5000/members/ (選擇任意一筆要修改資料的 _id值，貼在網址後面) 
3. Body -> form-data -> 填上要修改的資料 -> send
4. response -> updated data count: 1 (回傳要修改資料的筆數)

![update_db_members_id](img/update_db_members_id.png)